In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [3]:
# === МОДЕЛИ ПОЛА И ВОЗРАСТА ===
GENDER_PROTO = "models/gender_deploy.prototxt"
GENDER_MODEL = "models/gender_net.caffemodel"

AGE_PROTO = "models/age_deploy.prototxt"
AGE_MODEL = "models/age_net.caffemodel"

gender_net = cv2.dnn.readNetFromCaffe(GENDER_PROTO, GENDER_MODEL)
age_net = cv2.dnn.readNetFromCaffe(AGE_PROTO, AGE_MODEL)

MODEL_MEAN_VALUES = (78.4263, 87.7689, 114.8958)
GENDER_LIST = ['Male', 'Female']
AGE_BUCKETS = [
    '(0-2)', '(4-6)', '(8-12)', '(15-20)',
    '(25-32)', '(38-43)', '(48-53)', '(60-80)', '(80-100)'
]

# === MEDIAPIPE ===
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_face_detection = mp.solutions.face_detection

In [4]:
def draw_label(frame, landmarks, landmark_enum, label_text: str,
               visibility_thr: float, img_w: int, img_h: int,
               point_in_face_func):
    """
    Подписываем сустав, если он видим и не попадает в прямоугольник лица.
    """
    lm = landmarks[landmark_enum.value]
    if lm.visibility < visibility_thr:
        return

    x = int(lm.x * img_w)
    y = int(lm.y * img_h)

    if point_in_face_func(x, y):
        return

    cv2.putText(
        frame,
        label_text,
        (x + 5, y - 5),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (255, 255, 0),
        1,
        cv2.LINE_AA
    )


In [5]:


# === КАМЕРА === #
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Камера не открылась. Проверь доступ к камере.")
    raise SystemExit

with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=1,
        enable_segmentation=False,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.7) as pose, \
        mp_face_detection.FaceDetection(
            model_selection=0,  # 0 — ближние лица
            min_detection_confidence=0.7
        ) as face_det:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        h, w, _ = frame.shape

        # Перевод в RGB (нужно и для pose, и для face)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # -------- ЛИЦО (mediapipe) --------
        faces = []
        face_results = face_det.process(image_rgb)
        if face_results.detections:
            for det in face_results.detections:
                bbox = det.location_data.relative_bounding_box
                x1 = int(bbox.xmin * w)
                y1 = int(bbox.ymin * h)
                x2 = int((bbox.xmin + bbox.width) * w)
                y2 = int((bbox.ymin + bbox.height) * h)

                # ограничиваем внутри кадра
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(w - 1, x2)
                y2 = min(h - 1, y2)

                if x2 > x1 and y2 > y1:
                    faces.append((x1, y1, x2, y2))


        # функция: точка попадает внутрь какого-либо лица?
        def point_in_face(xp, yp) -> bool:
            for (fx1, fy1, fx2, fy2) in faces:
                if fx1 <= xp <= fx2 and fy1 <= yp <= fy2:
                    return True
            return False


        # -------- ПОЗА (скелет) --------
        result = pose.process(image_rgb)

        if result.pose_landmarks:
            landmarks = result.pose_landmarks.landmark
            visibility_thr = 0.7  # фильтр по видимости точки

            # Рисуем кости вручную, пропуская те, что заходят в область лица
            for connection in mp_pose.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]

                lm_start = landmarks[start_idx]
                lm_end = landmarks[end_idx]

                if lm_start.visibility < visibility_thr or lm_end.visibility < visibility_thr:
                    continue

                x1 = int(lm_start.x * w)
                y1 = int(lm_start.y * h)
                x2 = int(lm_end.x * w)
                y2 = int(lm_end.y * h)

                # Если хотя бы одна точка кости в области лица — не рисуем
                if point_in_face(x1, y1) or point_in_face(x2, y2):
                    continue

                cv2.line(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)

            # Подписи суставов (тоже с учётом лица)
            draw_label(frame, landmarks, mp_pose.PoseLandmark.LEFT_SHOULDER,
                       "Left shoulder", visibility_thr, w, h, point_in_face)
            draw_label(frame, landmarks, mp_pose.PoseLandmark.RIGHT_SHOULDER,
                       "Right shoulder", visibility_thr, w, h, point_in_face)

            draw_label(frame, landmarks, mp_pose.PoseLandmark.LEFT_ELBOW,
                       "Left elbow", visibility_thr, w, h, point_in_face)
            draw_label(frame, landmarks, mp_pose.PoseLandmark.RIGHT_ELBOW,
                       "Right elbow", visibility_thr, w, h, point_in_face)

            draw_label(frame, landmarks, mp_pose.PoseLandmark.LEFT_WRIST,
                       "Left hand", visibility_thr, w, h, point_in_face)
            draw_label(frame, landmarks, mp_pose.PoseLandmark.RIGHT_WRIST,
                       "Right hand", visibility_thr, w, h, point_in_face)

            draw_label(frame, landmarks, mp_pose.PoseLandmark.LEFT_KNEE,
                       "Left knee", visibility_thr, w, h, point_in_face)
            draw_label(frame, landmarks, mp_pose.PoseLandmark.RIGHT_KNEE,
                       "Right knee", visibility_thr, w, h, point_in_face)

            draw_label(frame, landmarks, mp_pose.PoseLandmark.LEFT_ANKLE,
                       "Left leg", visibility_thr, w, h, point_in_face)
            draw_label(frame, landmarks, mp_pose.PoseLandmark.RIGHT_ANKLE,
                       "Right leg", visibility_thr, w, h, point_in_face)

        # -------- ЛИЦО + ПОЛ + ВОЗРАСТ (через Caffe) --------
        for (x1, y1, x2, y2) in faces:
            face_crop = frame[y1:y2, x1:x2]
            if face_crop.size == 0:
                continue

            blob = cv2.dnn.blobFromImage(
                face_crop,
                1.0,
                (227, 227),
                MODEL_MEAN_VALUES,
                swapRB=False
            )

            # пол
            gender_net.setInput(blob)
            gPreds = gender_net.forward()
            gender = GENDER_LIST[np.argmax(gPreds[0])]
            g_conf = float(np.max(gPreds[0]))

            # возраст
            age_net.setInput(blob)
            aPreds = age_net.forward()
            age = AGE_BUCKETS[np.argmax(aPreds[0])]
            a_conf = float(np.max(aPreds[0]))

            # рамка и подпись
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(
                frame,
                f"{gender} {g_conf:.2f}, {age} {a_conf:.2f}",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 255, 0),
                2
            )

        cv2.imshow("Pose + Face + Gender + Age", frame)

        # выход по 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1763639834.466901 2066645 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4
I0000 00:00:1763639834.472523 2066645 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M4
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1763639834.489700 2074516 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/alexhoudini/PycharmProjects/pizda/.venv/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
W0000 00:00:1763639834.563793 2074501 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 